In [1]:
import pandas as pd
import numpy as np

In [2]:
# load data
df = pd.read_csv("../data/combine.csv")
df.head()

,Date,Close,ma_5,ma_10,ma_20,ma_30,ma_60,ma_90,ma_180,ma_240,...,Nasdaq_diff,SP500_diff,INDU_diff,Oil_diff,Btc_diffpct,Gold_diffpct,Nasdaq_diffpct,SP500_diffpct,INDU_diffpct,Oil_diffpct
0,2012-01-03,5.32,4.975,4.975000,4.975000,4.975000,4.975000,4.975000,4.975000,4.975000,...,0.00,0.00,0.00,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2012-01-04,5.57,5.094,5.094000,5.094000,5.094000,5.094000,5.094000,5.094000,5.094000,...,-0.36,0.24,21.04,0.8793,0.046992,0.014944,-0.000136,0.000188,0.001697,0.019997
2,2012-01-05,6.65,5.508,5.353333,5.353333,5.353333,5.353333,5.353333,5.353333,5.353333,...,21.50,3.76,-2.72,0.0000,0.193896,-0.004148,0.008118,0.002944,-0.000219,0.000000
3,2012-01-06,6.90,5.888,5.574286,5.574286,5.574286,5.574286,5.574286,5.574286,5.574286,...,4.36,-3.25,-55.78,0.0000,0.037594,-0.009996,0.001633,-0.002537,-0.004493,0.000000
4,2012-01-09,6.90,6.850,5.972000,5.972000,5.972000,5.972000,5.972000,5.972000,5.972000,...,2.34,2.89,32.77,0.0000,0.000000,0.003576,0.000875,0.002262,0.002651,0.000000


In [3]:
# define to check if close price increased compare to the previous close
df["Trend"] = df.apply(lambda x: "up" if x["Btc_diff"] > 0 else ("down" if x["Btc_diff"] <0 else "nc"), axis=1)

In [4]:
df[["Date", "Trend"]]

,Date,Trend
0,2012-01-03,nc
1,2012-01-04,up
2,2012-01-05,up
3,2012-01-06,up
4,2012-01-09,nc
...,...,...
2318,2021-03-25,down
2319,2021-03-26,up
2320,2021-03-29,up
2321,2021-03-30,up


In [5]:
ind_train_start = 360
ind_test_start = int((len(df) - 360) * 0.8)

In [6]:
df.columns

Index(['Date', 'Close', 'ma_5', 'ma_10', 'ma_20', 'ma_30', 'ma_60', 'ma_90',
       'ma_180', 'ma_240', 'ma_360', 'Gold', 'Nasdaq_Comp', 'SP500', 'INDU',
       'Oil', 'Btc_diff', 'Gold_diff', 'Nasdaq_diff', 'SP500_diff',
       'INDU_diff', 'Oil_diff', 'Btc_diffpct', 'Gold_diffpct',
       'Nasdaq_diffpct', 'SP500_diffpct', 'INDU_diffpct', 'Oil_diffpct',
       'Trend'],
      dtype='object')

In [7]:
# change here
X = df[['ma_5', 'ma_10', 'ma_20', 'ma_30', 'ma_60', 'ma_90', 'ma_180', 'ma_240', 'ma_360']]
y = df[["Close"]]

In [8]:
X_train = X.iloc[ind_train_start:ind_test_start]
X_test = X.iloc[ind_test_start:]
X_test = X_test.iloc[:len(X_test)-1]
X_test.iloc[0]

# for using close price
#y_train = y.iloc[ind_train_start + 1:ind_test_start + 1]
#y_test = y.iloc[ind_test_start+1:]
# for using other rows
y_train = y.iloc[ind_train_start+1:ind_test_start+1]
y_test = y.iloc[ind_test_start+1:]

In [9]:
print(X_train.shape, y_train.shape)

(1210, 9) (1210, 1)


In [10]:
ind_test = ind_test_start + 3
ind_test2 = ind_test - ind_test_start
print(df[["Date", "Close", 'Gold', 'Nasdaq_Comp', 'SP500', 'INDU', 'Oil', "Btc_diff", "Trend"]].iloc[ind_test])
print(X_test.iloc[ind_test2])
print(y_test.iloc[ind_test2])

Date           2018-04-10
Close              6880.0
Gold                12.77
Nasdaq_Comp        7094.3
SP500             2656.87
INDU              24408.0
Oil               18.5393
Btc_diff          -289.99
Trend                down
Name: 1573, dtype: object
ma_5       7010.556000
ma_10      7104.993000
ma_20      7699.797500
ma_30      8111.550333
ma_60      9297.584667
ma_90     10013.992444
ma_180    10563.570444
ma_240     9012.275833
ma_360     6798.611028
Name: 1573, dtype: float64
Close    6962.25
Name: 1574, dtype: float64


In [11]:
from sklearn.preprocessing import MinMaxScaler

In [12]:
X_scaler = MinMaxScaler().fit(X)
# for predict price
Y_scaler = MinMaxScaler().fit(y)

In [13]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [14]:
# for predict price
y_train_scaled = Y_scaler.transform(y_train)
y_test_scaled = Y_scaler.transform(y_test)

In [15]:
#y_train_scaled_shape = np.reshape(y_train_scaled, (y_train_scaled.shape[0]))
#print(y_train_scaled_shape.shape)

In [16]:
print(X_train_scaled, y_train_scaled)

[[0.00178074 0.00189652 0.00203574 ... 0.00204058 0.00188962 0.00169488]
 [0.00175355 0.00186667 0.00202467 ... 0.00205743 0.00190502 0.00170874]
 [0.00174013 0.00183887 0.00200622 ... 0.0020739  0.00192001 0.00172228]
 ...
 [0.12186884 0.13256474 0.14332164 ... 0.3266061  0.33139138 0.31795718]
 [0.12035115 0.13004816 0.1417343  ... 0.3271523  0.33204511 0.31879345]
 [0.12084278 0.12795257 0.14087676 ... 0.32767387 0.33266576 0.31961661]] [[0.00175706]
 [0.00171463]
 [0.00162415]
 ...
 [0.12384775]
 [0.12253086]
 [0.11416246]]


In [17]:
print(X_train_scaled.shape, y_train_scaled.shape)

(1210, 9) (1210, 1)


In [18]:
y_reshape = np.reshape(y_train_scaled, (y_train_scaled.shape[0]))

In [19]:
from sklearn.svm import SVR 
model = SVR(kernel='rbf')
model

SVR()

In [20]:
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10, 50, 100, 1000],
              'gamma': [0.00001, 0.0005, 0.0001, 0.0005, 0.0001]}
grid = GridSearchCV(model, param_grid, verbose=3)

In [21]:
grid.fit(X_train_scaled, y_reshape)

Fitting 5 folds for each of 60 candidates, totalling 300 fits
[CV 1/5] END ...........................C=0.001, gamma=1e-05; total time=   0.0s
[CV 2/5] END ...........................C=0.001, gamma=1e-05; total time=   0.0s
[CV 3/5] END ...........................C=0.001, gamma=1e-05; total time=   0.0s
[CV 4/5] END ...........................C=0.001, gamma=1e-05; total time=   0.0s
[CV 5/5] END ...........................C=0.001, gamma=1e-05; total time=   0.0s
[CV 1/5] END ..........................C=0.001, gamma=0.0005; total time=   0.0s
[CV 2/5] END ..........................C=0.001, gamma=0.0005; total time=   0.0s
[CV 3/5] END ..........................C=0.001, gamma=0.0005; total time=   0.0s
[CV 4/5] END ..........................C=0.001, gamma=0.0005; total time=   0.0s
[CV 5/5] END ..........................C=0.001, gamma=0.0005; total time=   0.0s
[CV 1/5] END ..........................C=0.001, gamma=0.0001; total time=   0.0s
[CV 2/5] END ..........................C=0.001,

[CV 2/5] END .............................C=0.5, gamma=1e-05; total time=   0.0s
[CV 3/5] END .............................C=0.5, gamma=1e-05; total time=   0.0s
[CV 4/5] END .............................C=0.5, gamma=1e-05; total time=   0.0s
[CV 5/5] END .............................C=0.5, gamma=1e-05; total time=   0.0s
[CV 1/5] END ............................C=0.5, gamma=0.0005; total time=   0.0s
[CV 2/5] END ............................C=0.5, gamma=0.0005; total time=   0.0s
[CV 3/5] END ............................C=0.5, gamma=0.0005; total time=   0.0s
[CV 4/5] END ............................C=0.5, gamma=0.0005; total time=   0.0s
[CV 5/5] END ............................C=0.5, gamma=0.0005; total time=   0.0s
[CV 1/5] END ............................C=0.5, gamma=0.0001; total time=   0.0s
[CV 2/5] END ............................C=0.5, gamma=0.0001; total time=   0.0s
[CV 3/5] END ............................C=0.5, gamma=0.0001; total time=   0.0s
[CV 4/5] END ...............

[CV 2/5] END .............................C=50, gamma=0.0005; total time=   0.0s
[CV 3/5] END .............................C=50, gamma=0.0005; total time=   0.0s
[CV 4/5] END .............................C=50, gamma=0.0005; total time=   0.0s
[CV 5/5] END .............................C=50, gamma=0.0005; total time=   0.0s
[CV 1/5] END .............................C=50, gamma=0.0001; total time=   0.0s
[CV 2/5] END .............................C=50, gamma=0.0001; total time=   0.0s
[CV 3/5] END .............................C=50, gamma=0.0001; total time=   0.0s
[CV 4/5] END .............................C=50, gamma=0.0001; total time=   0.0s
[CV 5/5] END .............................C=50, gamma=0.0001; total time=   0.0s
[CV 1/5] END .............................C=100, gamma=1e-05; total time=   0.0s
[CV 2/5] END .............................C=100, gamma=1e-05; total time=   0.0s
[CV 3/5] END .............................C=100, gamma=1e-05; total time=   0.0s
[CV 4/5] END ...............

GridSearchCV(estimator=SVR(),
             param_grid={'C': [0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10, 50,
                               100, 1000],
                         'gamma': [1e-05, 0.0005, 0.0001, 0.0005, 0.0001]},
             verbose=3)

In [22]:
print(grid.best_params_)
print(grid.best_score_)

{'C': 0.001, 'gamma': 0.0005}
-1429.5585374297282
